In [1]:
!pip install -q python-louvain networkx matplotlib seaborn pandas numpy pycountry bertopic sentence-transformers umap-learn hdbscan
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import os
import re
import community as community_louvain  # python-louvain
import pycountry
from itertools import combinations
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import community.community_louvain as community_louvain

import re, pandas as pd, numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import combinations
import community.community_louvain as community_louvain
import seaborn as sns
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import umap
import hdbscan
import numpy as np


from collections import Counter
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings("ignore")

## Keyword over Time

In [2]:
url = "https://raw.githubusercontent.com/namigabbasov/ai-literacy-bibliometric-analysis/refs/heads/main/data/ai_literacy.csv"
df = pd.read_csv(url, low_memory=False)

# Ensure Year is numeric
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.dropna(subset=["Year"])
df["Year"] = df["Year"].astype(int)


def normalize_keywords(text):
    if pd.isna(text):
        return []
    tokens = re.split(r';|,|\|', str(text))
    return [t.strip().lower() for t in tokens if t.strip() not in ['', 'nan']]

# Combine author + index keywords
df['keywords_combined'] = df['Author Keywords'].apply(normalize_keywords) + \
                          df['Index Keywords'].apply(normalize_keywords)

# Noise keywords (generic metadata)
noise_keywords = {
    "female","male","human","humans",
    "questionnaire","surveys","survey",
    "systematic review","review","article","method", "students", "adult", "teaching"
}

df['keywords_clean'] = df['keywords_combined'].apply(
    lambda kws: [kw for kw in kws if kw not in noise_keywords]
)

df_kw = df[df['keywords_clean'].apply(len) > 0].copy()

all_keywords = [kw for kws in df_kw['keywords_clean'] for kw in kws]
keyword_counts = Counter(all_keywords)

df_keywords = (
    pd.DataFrame(keyword_counts.items(), columns=["keyword", "count"])
    .sort_values("count", ascending=False)
)

top_keywords = df_keywords.head(1000).copy()
top_keywords_list = top_keywords["keyword"].tolist()

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(top_keywords_list, show_progress_bar=True)

NUM_CLUSTERS = 10
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
clusters = kmeans.fit_predict(embeddings)

top_keywords["cluster"] = clusters

rows = []
for _, row in df_kw.iterrows():
    year = row["Year"]
    for kw in row["keywords_clean"]:
        if kw in top_keywords_list:
            rows.append((year, kw))

df_kw_year = pd.DataFrame(rows, columns=["Year", "keyword"])
df_kw_year = df_kw_year.groupby(["Year", "keyword"]).size().reset_index(name="count")
df_kw_year = df_kw_year.merge(top_keywords[["keyword", "cluster"]], on="keyword")


cluster_summary = (
    top_keywords.groupby("cluster")
    .apply(lambda g: g.sort_values("count", ascending=False).head(10)[["keyword","count"]])
)
print("\n===== CLUSTER SUMMARY (TOP 10 PER CLUSTER) =====\n")
print(cluster_summary)

auto_labels = {
    c: top_keywords[top_keywords["cluster"] == c]
       .sort_values("count", ascending=False)
       .iloc[0]["keyword"].title()
    for c in sorted(top_keywords["cluster"].unique())
}

cluster_label_map = {c: auto_labels[c] for c in auto_labels}

top_keywords["cluster_label"] = top_keywords["cluster"].map(cluster_label_map)
df_kw_year["cluster_label"] = df_kw_year["cluster"].map(cluster_label_map)

cluster_growth = {}

for c in sorted(top_keywords["cluster"].unique()):
    agg = (
        df_kw_year[df_kw_year["cluster"] == c]
        .groupby("Year")["count"]
        .sum()
        .reset_index()
        .sort_values("Year")
    )
    if len(agg) > 1:
        slope = np.polyfit(agg["Year"], agg["count"], 1)[0]
    else:
        slope = 0
    cluster_growth[c] = slope

print("\n===== CLUSTER GROWTH SLOPES =====\n")
for c, sl in cluster_growth.items():
    print(f"Cluster {c} ({cluster_label_map[c]}): slope = {sl:.2f}")

clusters_sorted = sorted(top_keywords["cluster"].unique())

fig, axes = plt.subplots(2, 5, figsize=(20, 8), sharex=True, sharey=True, dpi=1000)
axes = axes.flatten()

for idx, c in enumerate(clusters_sorted):
    ax = axes[idx]

    cluster_df = df_kw_year[df_kw_year["cluster"] == c].copy()
    cluster_df = cluster_df.sort_values("Year")

    # Grey keyword lines
    for kw in cluster_df["keyword"].unique():
        sub = cluster_df[cluster_df["keyword"] == kw].sort_values("Year")
        ax.plot(sub["Year"], sub["count"], color="lightgrey", linewidth=1)

    # Red cluster aggregate
    agg = (
        cluster_df.groupby("Year")["count"].sum().reset_index().sort_values("Year")
    )
    sns.lineplot(data=agg, x="Year", y="count", ax=ax,
                 color="red", linewidth=2)

    ax.set_title(cluster_label_map[c], fontsize=14, weight="bold")
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.tight_layout()
plt.show()


Batches:   0%|          | 0/32 [00:00<?, ?it/s]


===== CLUSTER SUMMARY (TOP 10 PER CLUSTER) =====

                                    keyword  count
cluster                                           
0       483         sustainable development     25
        684                  sustainability     11
        3917     entrepreneurship education      8
        3471                         equity      6
        3538  sustainable development goals      6
...                                     ...    ...
9       661            contrastive learning     57
        323           large language models     56
        93                     data privacy     38
        126                   deep learning     33
        1621    natural language processing     28

[100 rows x 2 columns]

===== CLUSTER GROWTH SLOPES =====

Cluster 0 (Sustainable Development): slope = 7.03
Cluster 1 (Artificial Intelligence): slope = 238.75
Cluster 2 (Engineering Education): slope = 103.64
Cluster 3 (Higher Education): slope = 55.19
Cluster 4 (Nursing): slope = 7

## BERTopic

In [4]:
def clean_ai_literacy_text(text):
    if pd.isna(text):
        return ""
    t = text

    # Remove all copyright/Elsevier/publisher boilerplate
    t = re.sub(r"©.*?(All rights reserved\.?)", " ", t, flags=re.DOTALL|re.IGNORECASE)
    t = re.sub(r"All rights reserved\.?", " ", t, flags=re.IGNORECASE)
    t = re.sub(r"Elsevier[^.,;]*", " ", t, flags=re.IGNORECASE)
    t = re.sub(r"©", " ", t)

    # Remove AIED 2025 proceedings block (multi-line, huge block)
    t = re.sub(
        r"26th International Conference on Artificial Intelligence in Education.*?(?=©|$)",
        " ",
        t,
        flags=re.DOTALL | re.IGNORECASE
    )

    # Remove any conference boilerplate
    t = re.sub(
        r"International Conference on .*?(?=©|$)",
        " ",
        t,
        flags=re.DOTALL | re.IGNORECASE
    )

    # Remove repeated paper title lists separated by semicolons
    t = re.sub(r";\s*[A-Z].+?(?=©|$)", " ", t, flags=re.DOTALL)

    # Remove very long sequences with multiple semicolons
    t = re.sub(r"(;){2,}", ". ", t)

    # Remove extra whitespace
    t = re.sub(r"\s+", " ", t)

    return t.strip()


In [7]:
url = "https://raw.githubusercontent.com/namigabbasov/ai-literacy-bibliometric-analysis/refs/heads/main/data/ai_literacy.csv"
df = pd.read_csv(url, low_memory=False)
df["Title"] = df["Title"].fillna("").astype(str)
df["Abstract"] = df["Abstract"].fillna("").astype(str)


def clean_ai_literacy_text(text):
    if pd.isna(text):
        return ""

    t = text


    # Remove publisher copyright & boilerplate

    t = re.sub(r"©.*?(All rights reserved\.?)", " ", t, flags=re.DOTALL|re.IGNORECASE)
    t = re.sub(r"All rights reserved\.?", " ", t, flags=re.IGNORECASE)
    t = re.sub(r"Elsevier[^.,;]*", " ", t, flags=re.IGNORECASE)
    t = re.sub(r"©", " ", t)


    # Remove AIED & proceedings boilerplate

    t = re.sub(
        r"26th International Conference on Artificial Intelligence in Education.*?(?=$)",
        " ",
        t,
        flags=re.DOTALL | re.IGNORECASE
    )

    # Remove any conference proceedings descriptions
    t = re.sub(
        r"International Conference on .*?(?=$)",
        " ",
        t,
        flags=re.DOTALL | re.IGNORECASE
    )

    # SIGGRAPH educator forum, ACM, IEEE
    t = re.sub(
        r"Proceedings.*?(?=$)",
        " ",
        t,
        flags=re.DOTALL | re.IGNORECASE
    )


    # Remove long list of paper titles

    t = re.sub(r";\s*[A-Z].+?(?=$)", " ", t, flags=re.DOTALL)

    # Collapse whitespace
    t = re.sub(r"\s+", " ", t)

    return t.strip()



# APPLY CLEANING TO ALL DOCUMENTS


df["text_raw"] = df["Title"] + ". " + df["Abstract"]
df["clean_text"] = df["text_raw"].apply(clean_ai_literacy_text)


# DROP INVALID ABSTRACTS with proceedings front-matter only


def is_bad(text):
    bad_patterns = [
        "proceedings contain",
        "conference",
        "Proceedings",
        "SIGGRAPH",
        "Educator Forum",
        "AIED 2025",
        "Foreword",
        "Preface",
        "abstract foreword"
    ]
    return any(p.lower() in text.lower() for p in bad_patterns)

df_clean = df[~df["clean_text"].apply(is_bad)].copy()

# Drop super-short texts (less than 30 chars)
df_clean = df_clean[df_clean["clean_text"].str.len() > 50]

print("Original documents:", len(df))
print("Documents after cleaning:", len(df_clean))


# PREPARE DOCUMENTS FOR BERTOPIC


documents = df_clean["clean_text"].tolist()


# EMBEDDINGS
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedding_model.encode(
    documents,
    show_progress_bar=True,
    convert_to_numpy=True
)


# VECTORIZER
vectorizer_model = CountVectorizer(
    stop_words="english",
    min_df=2,
    max_df=0.95,
    ngram_range=(1, 2)
)

# REPRESENTATION MODEL

representation_model = KeyBERTInspired(top_n_words=12)


# UMAP + HDBSCAN

umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=5,
    min_dist=0.0,
    metric="cosine",
    random_state=42
)

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=20,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

# BUILD FINAL BERTOPIC MODEL

topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    representation_model=representation_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    top_n_words=12,
    min_topic_size=20,
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(documents, embeddings)


# INSPECT TOPICS
topic_info = topic_model.get_topic_info()
topic_info


Original documents: 2227
Documents after cleaning: 2149


Batches:   0%|          | 0/68 [00:00<?, ?it/s]

2025-12-02 02:25:23,603 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-02 02:25:44,807 - BERTopic - Dimensionality - Completed ✓
2025-12-02 02:25:44,809 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-02 02:25:44,978 - BERTopic - Cluster - Completed ✓
2025-12-02 02:25:44,983 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-02 02:25:46,757 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,0,1860,0_teaching ai_intelligence education_ai higher...,"[teaching ai, intelligence education, ai highe...",[A Holistic Understanding of Chinese Children'...
1,1,211,1_literacy attitudes_education artificial_conc...,"[literacy attitudes, education artificial, con...",[Nursing students' attitudes and literacy towa...
2,2,78,2_ai engagement_education aigc_learning resear...,"[ai engagement, education aigc, learning resea...",[An initial interpretation of the U.S. Departm...
